In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这个笔记本是从[Rajesh Thallam](https://github.com/RajeshThallam/vertex-ai-labs/blob/main/07-vertex-train-deploy-lightgbm/vertex-train-deploy-lightgbm-model.ipynb)的笔记本的修订版。

# GCP 上的 E2E 机器学习：MLOps 阶段2：实验：使用 Vertex AI Training 开始使用 LightGBM

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看 GitHub 上的页面
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps生产环境。本教程涵盖了阶段2：实验阶段：使用Vertex AI 训练LightGBM 模型的入门指南。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Training` 训练一个 LightGBM 自定义模型。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源

执行的步骤包括：

- 使用 Python 包进行训练。
- 使用 GCSFuse 将模型工件保存到 Cloud Storage。
- 构建一个 FastAPI 预测服务器。
- 构建一个 Dockerfile 部署镜像。
- 在本地测试部署镜像。
- 创建一个 `Vertex AI Model` 资源。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。该数据集不需要任何特征工程。本教程中数据集的版本存储在一个公共的云存储桶中。训练好的模型可以预测三种不同种类的鸢尾花品种：山鸢尾(setosa)、弗吉尼亚鸢尾(virginica)和变色鸢尾(versicolor)。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- Cloud Storage SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Cloud Storage指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供满足这些要求的详细说明。以下步骤提供了一套简洁的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，可以在终端shell中运行 `pip3 install jupyter`。

5. 要启动Jupyter，可以在终端shell中运行 `jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开这个笔记本。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install -U google-cloud-storage $USER_FLAG -q
! pip3 install -U lightgbm $USER_FLAG -q

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade tensorflow $USER_FLAG -q

重新启动内核

安装了额外的程序包后，您需要重新启动笔记本内核，以便它可以找到这些程序包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，则需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK对该笔记本中的所有命令使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并插入以`$`为前缀的Python变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶来训练Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

if REGION == "[your-region]":
    REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会议，则可能会使用共享的测试帐户或项目。为避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下方的单元格，并按照提示进行OAuth验证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，进入[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入一个名称，然后点击**创建**。

在**将此服务账号授权给项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage对象管理员"，并选择**Storage对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下方的单元格中将您的服务账号密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源关联的所有数据在不同会话之间保留的位置。

请在下面设置您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织以外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只要您的存储桶还不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import lightgbm

In [ ]:
print(f"LightGBM version {lightgbm.__version__}")

## 初始化Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 启用 Artifact Registry API

首先，您必须为您的项目启用 Artifact Registry API 服务。

了解有关[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)的更多信息。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### 创建一个私有的Docker仓库

你的第一步是在Google Artifact Registry中创建你自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令来创建一个新的Docker仓库，指定你的地区，并附上描述"docker repository"。

2. 运行`gcloud artifacts repositories list`命令来验证你的仓库是否已经创建成功。

In [ ]:
PRIVATE_REPO = "prediction"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Prediction repository"

! gcloud artifacts repositories list

### 配置私有仓库的身份验证

在推送或拉取容器映像之前，请配置Docker使用`gcloud`命令行工具对您所在地区的`Artifact Registry`发出的请求进行身份验证。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

#### 设置预构建的容器

设置用于训练和预测的预构建的 Docker 容器镜像。


有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "lightgbm-cpu"

# prebuilt
TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

DEPLOY_IMAGE = "{}-docker.pkg.dev/{}/{}/{}:latest".format(
    REGION, PROJECT_ID, PRIVATE_REPO, DEPLOY_VERSION
)
print("Deploy image:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB的内存。
     - `n1-highmem`: 每个vCPU 6.5GB的内存
     - `n1-highcpu`: 每个vCPU 0.9GB的内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \]个数

*注意：以下机型不支持用于训练*

 - `标准`: 2个vCPUs
 - `高CPU`: 2、4和8个vCPUs

*注意：您也可以使用n2和e2机型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查训练包

#### 包布局

在您开始训练之前，您将查看一个用于定制训练作业的Python包是如何组装的。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker映像的操作环境中的指令。

文件`trainer/task.py`是执行定制训练作业的Python脚本。*注意*，当我们在工作池规范中引用它时，我们用句点（`trainer.task`）替换目录斜杠，并删除文件后缀（`.py`）。

#### 包装配

在接下来的单元中，您将组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n'lightgbm'    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python培训包创建任务脚本

接下来，您需要为训练包创建`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-dir`：保存训练模型的位置。在使用Vertex AI自定义训练时，该位置将在环境变量`AIP_MODEL_DIR`中指定。
    
- 数据预处理（`get_data()`）：
    - 下载数据集并拆分为训练和测试集。
    
- 训练（`train_model()`）：
    - 训练模型。
    
- 模型结果保存
    - 将模型结果和评估指标保存在由`model-dir`指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

import lightgbm as lgb

import argparse
import logging

logging.getLogger().setLevel(logging.INFO)

logging.info("Parsing arguments")

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model-dir', 
    dest='model_dir',        
    default=os.getenv('AIP_MODEL_DIR'), 
    type=str, 
    help='Location to export GCS model')
args = parser.parse_args()
logging.info(args)

def get_data():
    # Download data
    logging.info("Downloading data")
    iris = load_iris()
    print(iris.data.shape)

    # split data
    print("Splitting data into test and train")
    x, y = iris.data, iris.target
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

    # create dataset for lightgbm
    print("creating dataset for LightGBM")
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
    
    return lgb_train, lgb_eval

def train_model(lgb_train, lg_eval):
    # specify your configurations as a dict
    params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_error'},
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
        'num_class' : 3
    }

    # train lightgbm model
    logging.info('Starting training...')
    model = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5)
    
    return model

lgb_train, lgb_eval = get_data()
model = train_model(lgb_train, lgb_eval)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)
        
# save model to file
logging.info('Saving model...')
model_filename = 'model.txt'
gcs_model_path = os.path.join(args.model_dir, model_filename)
model.save_model(gcs_model_path)

将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

### 创建和运行自定义训练任务

为了训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练任务，2) 运行任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类来创建一个自定义训练任务，并指定以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball格式存储在云存储中。
- `python_module_name`：Python包中训练脚本的相对路径。

*注意：* 没有`requirements`参数。您可以在Python包中的`setup.py`脚本中指定任何需求。

In [ ]:
DISPLAY_NAME = "iris_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_iris.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    project=PROJECT_ID,
)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型文件的位置。
      - 直接：您将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型文件位置。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = []

运行自定义训练作业

接下来，您可以通过调用`run`方法来运行自定义作业，启动训练作业，具体参数如下：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `base_output_dir`：用于写入模型工件的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
job.run(
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    base_output_dir=MODEL_DIR,
    sync=False,
)

model_path_to_deploy = MODEL_DIR + "/model"

### 列出一个自定义培训工作

In [ ]:
_job = job.list(filter=f"display_name=iris_{TIMESTAMP}")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻止直到自定义训练作业完成。

In [ ]:
job.wait()

### 删除自定义训练作业

在训练作业完成之后，您可以使用 `delete()` 方法来删除训练作业。在完成之前，可以使用 `cancel()` 方法取消训练作业。

In [ ]:
job.delete()

### 验证模型工件

接下来，验证训练脚本是否成功将训练好的模型保存到您的云存储位置。

In [ ]:
print(f"Model path with trained model artifacts {model_path_to_deploy}")

! gsutil ls $model_path_to_deploy

## 部署 LightGBM 模型到 Vertex AI 端点

### 使用 FastAPI 构建 HTTP 服务器

接下来，您可以使用 FastAPI 来实现一个 HTTP 服务器作为自定义部署容器。该容器必须监听并响应存活检查、健康检查和预测请求。HTTP 服务器必须在 0.0.0.0 上监听请求。

了解更多关于[部署容器要求](https://cloud.google.com/ai-platform-unified/docs/predictions/custom-container-requirements#image)。

了解更多关于[FastAPI](https://fastapi.tiangolo.com/)。

In [ ]:
# Make folder for serving script
! rm -rf serve
! mkdir serve

# Make the predictor subfolder
! mkdir serve/app

为服务容器创建要求文件

接下来，在服务器环境中创建`requirements.txt`文件，指定需要在服务容器上安装的Python包。

In [ ]:
%%writefile serve/requirements.txt

numpy
scikit-learn>=0.24
pandas==1.0.4
lightgbm==3.2.1
google-cloud-storage>=1.26.0,<2.0.0dev

### 写一个FastAPI的服务脚本

接下来，您可以使用`FastAPI`编写HTTP服务器的服务脚本，如下所示：

- `app`: 实例化一个`FastAPI`应用程序
- `health()`: 定义对健康请求的响应。
    - 返回状态码200
- `predict()`: 定义对预测请求的响应。
    - `body = await request.json()`: 异步等待HTTP请求。
    - `instances = body["instances"]`: 预测请求的内容。
    - `inputs = np.asarray(instances)`: 重新格式化预测请求为一个numpy数组。
    - `outputs = model.predict(inputs)`: 调用模型进行预测。
    - `return {"predictions": ... }`: 在响应体中返回格式化后的预测结果。

In [ ]:
%%writefile serve/app/main.py
from fastapi.logger import logger
from fastapi import FastAPI, Request

import numpy as np
import os

from sklearn.datasets import load_iris
import lightgbm as lgb

import logging

gunicorn_logger = logging.getLogger('gunicorn.error')
logger.handlers = gunicorn_logger.handlers

if __name__ != "main":
    logger.setLevel(gunicorn_logger.level)
else:
    logger.setLevel(logging.DEBUG)

app = FastAPI()

model_f = "/model/model.txt"

logger.info("Loading model")
_model = lgb.Booster(model_file=model_f)
logger.info("Loading target class labels")
_class_names = load_iris().target_names

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    """ health check to ensure HTTP server is ready to handle 
        prediction requests
    """
    return {"status": "healthy"}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    
    outputs = _model.predict(inputs)

    logger.info(f"Outputs {outputs}")
    return {"predictions": [_class_names[class_num] for class_num in np.argmax(outputs, axis=1)]}

### 添加预启动脚本
FastAPI将在启动服务器之前执行此脚本。为了在与AI平台（Unified）所期望的端口上运行FastAPI，将`PORT`环境变量设置为等于`AIP_HTTP_PORT`。

In [ ]:
%%writefile serve/app/prestart.sh

#!/bin/bash
export PORT=$AIP_HTTP_PORT

### 存储测试实例

接下来，您创建合成示例，以后可用于测试 FastAPI 服务器和训练好的 LightGBM 模型。

了解有关[自定义模型预测请求的 JSON 格式化](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)。

In [ ]:
%%writefile serve/instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

构建并推送预测容器到Artifact Registry

编写Dockerfile，使用`tiangolo/uvicorn-gunicorn-fastapi`作为基础镜像。这将自动运行FastAPI，使用Gunicorn和Uvicorn。

了解更多关于[使用Docker部署FastAPI](https://fastapi.tiangolo.com/deployment/docker/)。

In [ ]:
%%bash -s $MODEL_DIR

MODEL_DIR=$1

mkdir -p ./serve/model/
gsutil cp -r ${MODEL_DIR}/model/ ./serve/model/ 

cat > ./serve/Dockerfile <<EOF
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY ./model /
COPY requirements.txt requirements.txt

RUN pip3 install -r requirements.txt

EXPOSE 7080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "7080"]

EOF

构建本地容器

接下来，您可以构建并标记您的自定义部署容器。

In [ ]:
if not IS_COLAB:
    ! docker build --tag={DEPLOY_IMAGE} ./serve
else:
    # install docker daemon
    ! apt-get -qq install docker.io

### 在本地运行和测试容器（可选）

以分离模式在本地运行容器，并提供容器所需的环境变量。这些环境变量将在部署后由AI平台预测提供给容器。测试 `/health` 和 `/predict` 路由，然后停止运行中的映像。

在将容器映像推送到Artifact Registry以与Vertex Predictions一起使用之前，您可以在本地环境中将其作为容器运行，以验证服务器是否按预期工作。

In [ ]:
if not IS_COLAB:
    ! docker rm local-iris 2>/dev/null
    ! docker run -t -d --rm -p 7080:7080 \
        --name=local-iris \
        -e AIP_HTTP_PORT=7080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={MODEL_DIR} \
        {DEPLOY_IMAGE}
    ! docker container ls
    ! sleep 10

健康检查

发送容器服务器健康检查。输出应为 {"status": "healthy"}。

In [ ]:
if not IS_COLAB:
    ! curl http://localhost:7080/health

如果成功，服务器将返回以下响应：

```
{
   "status": "healthy"
}
```

预测检查

向容器的服务器发送一个预测请求。

In [ ]:
! curl -X POST \
  -d @serve/instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  http://localhost:7080/predict

这个请求使用一个测试句子。如果成功，服务器会以下列格式返回预测结果：

```
{"predictions":["versicolor","setosa"]}
```

停止本地容器

最后，停止本地容器。

In [ ]:
if not IS_COLAB:
    ! docker stop local-iris

将提供容器推送到Artifact Registry

将带有推理代码和依赖项的容器镜像推送到Artifact Registry。

In [ ]:
if not IS_COLAB:
    ! docker push $DEPLOY_IMAGE

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build --tag={DEPLOY_IMAGE} ./serve
docker push $2
kill $(jobs -p)

将服务容器部署到Vertex Predictions
我们在Vertex 人工智能上创建一个模型资源，并将模型部署到一个Vertex 终端。在使用模型之前，您必须将模型部署到终端。部署的模型会运行自定义容器图像以提供预测。

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，使用以下参数：

- `display_name`：`Model`资源的人类可读名称。
- `artifact`：训练模型工件的云存储位置。
- `serving_container_image_uri`：提供服务的容器镜像。
- `serving_container_predict_route`：发送预测请求到容器的HTTP路径。
- `serving_container_health_route`：发送健康检查请求到容器的HTTP路径。
- `serving_container_ports`：容器暴露的端口，用于监听请求。
- `sync`：是否以异步或同步方式进行上传。

如果使用异步方式运行`upload()`方法，您随后可以使用`wait()`方法阻塞直到完成。

In [ ]:
APP_NAME = "iris"

model_display_name = f"{APP_NAME}-{TIMESTAMP}"
model_description = "LightGBM based iris flower classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = "/predict"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

进行批量预测

文档 - [批量预测请求 - Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项目

您将使用合成数据作为测试数据项。不必担心我们使用的是合成数据 -- 我们只是想要展示如何进行预测。

In [ ]:
INSTANCES = [[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]]

### 制作批量输入文件

现在制作一个批量输入文件，您将存储在本地的云存储桶中。预测请求中的每个实例都是以下形式的列表：

                        [ [内容1], [内容2] ]

- `内容`: 测试项的特征值作为列表。

In [ ]:
import json

[json.dumps(record) for record in INSTANCES]

In [ ]:
import tensorflow as tf

gcs_input_uri = f"{BUCKET_URI}/{APP_NAME}/test/batch_input/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法来发起批量预测，使用以下参数：

- `job_display_name`: 批量预测作业的可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的云存储位置。
- `instances_format`: 输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`: 用于训练的机器类型。
- `sync`: 如果设置为True，则在等待异步批量作业完成时，调用将会被阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name=f"{APP_NAME}_TIMESTAMP",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=f"{BUCKET_URI}/{APP_NAME}/test/batch_output/",
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批处理作业完成。或者，在`batch_predict()` 方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测任务中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud 存储输出桶中。您可以调用 iter_outputs() 方法获取生成的结果文件列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

做在线预测

文档 - [在线预测请求](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## 部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法，并传入以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：传入一个包含一个或多个键值对的字典，表示流量分配给该模型的百分比。
如果只有一个模型，可以指定为{ "0": 100 }，其中"0"表示已上传的模型，100表示100%的流量。
如果端点上已经存在其他模型，需要将流量进行分流，则使用model_id来指定：{ "0": 百分比, model_id: 百分比, ... }，其中model_id是已部署到端点的模型的模型ID。百分比必须加起来等于100。
- `machine_type`：用于训练的计算机类型。
- `starting_replica_count`：最初配置的计算实例数。
- `max_replica_count`：可扩展到的最大计算实例数。在本教程中，只配置了一个实例。

In [ ]:
DEPLOYED_NAME = f"{APP_NAME}-{TIMESTAMP}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

### 进行预测

现在您的 `Model` 资源已部署到 `Endpoint` 资源上，您可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于 predict() 方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从 predict() 调用的响应是一个带有以下条目的Python字典：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `predictions`: 每个类别标签的预测置信度，介于 0 到 1 之间。
- `deployed_model_id`: 执行预测的部署了 `Model` 资源的 Vertex AI 标识符。

In [ ]:
instances_list = INSTANCES

prediction = endpoint.predict(instances_list)
print(prediction)

## 撤销模型

当你完成了预测工作后，你可以从`Endpoint`资源中撤销模型。这将取消所有计算资源并终止已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除您的私人Docker仓库

最后，当您的私人仓库变得过时时，请使用命令`gcloud artifacts repositories delete`在`Google Artifact Registry`中将其删除。

In [ ]:
! gcloud artifacts repositories delete {PRIVATE_REPO} --location={REGION} --quiet

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于此教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
# Delete the model using the Vertex model object
try:
    model.delete()
    endpoint.delete()
except Exception as e:
    print(e)

delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI